<a href="https://colab.research.google.com/github/kit-battarbee/on_the_efficiency_mobs/blob/main/mobs_experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# module for generating automorphism

!pip install permutation

In [ ]:
import math 


from math import floor, sqrt
from random import shuffle, randint
from secrets import randbelow
from permutation import Permutation


In [ ]:
# generate random objects

def randstring(k):
    """to define random matrices we need to get random bitstrings, which are the elements of the matrices"""
    string = []
    for i in range(k):
        m = randint(0,1) ##source of randomness, run each time on iteration for fresh randomness i.e. each element of bitstring independently distributed
                         ##this is giving us true / false with p = 1/2
        if m == 0:
           string.append(False)
        if m == 1:
           string.append(True)
    return string  

def rand_bool_mat(n,k): 
    """generate a random n x n matrix over bitstrings length k, intepreted as nested lists of Boolean values"""
    matrix = []
    for i in range(n):
        matrix.append([]) ##initialise the matrix with n empty rows
        for j in range(n):
            matrix[i].append(randstring(k)) #in each empty row add n random bitstrings length k
    return matrix       


In [ ]:
# operations on bitstrings

def bitand(a,b):
    """run the and operation on two bitstrings component-wise"""
    product = []
    for i in range(len(a)): 
           j = a[i] and b[i]
           product.append(j)
    return product


def bitor(a,b):
    """runs the or operation on two bitstrings component-wise"""
    sum = []
    if len(a) != len(b):
       return 'Error'
    else:
       for i in range(len(a)): ## we already checked len(a) = len(b) so we can use either as the upper bound for indexing variable
           j = a[i] or b[i]
           sum.append(j)
    return sum




In [ ]:
# matrix multiplication

def bitdots(a,b):
    """to define matrix multiplication it is useful to have a notion of the dot product (note a matrix product element is effectively a row dot product a column); here 
       define a dot product w.r.t bitor, bitand"""
    d = bitand(a[0],b[0])  ##initial value of d for iteration
    for i in range(len(a)-1): ##up to len(a)-1 due to i+1 counter
        d = bitor(d, bitand(a[i+1],b[i+1]))
    return d

def col_mat(a,k):    
  
    col = []
    for i in range(len(a)):
        col.append(a[i][k]) ##appends our list with kth entry of every row, which is exactly the kth column
    return col

def prod_bool_mat(a,b):
    prod = []
    for i in range(len(a)):
        prod.append([]) ##intialise rows of product matrix
        for j in range(len(a)):
            prod[i].append(bitdots(a[i],col_mat(b,j)))
    return prod    

def sum_bool_mat(a, b):
    sum = []
    for i in range(len(a)):
        sum.append([])
        for j in range(len(a)):
            sum[i].append(bitor(a[i][j], b[i][j]))
    return sum




In [ ]:
# generate high-order bitstring permutation

def generate_permutation(size):
    
    shuffled_nums = [(i + 1) for i in range(size)]
    shuffle(shuffled_nums)
    
    if (size == 2 or size == 3):
        return Permutation.cycle(*shuffled_nums)
    else:
        bool_vals = [True for i in range(2, size)]
        subcycle_lengths = []
        
        j = 0
        while(sum(subcycle_lengths) < size and j < size - 2):
            if (bool_vals[j] == True):
                cur_val = 2
                while ((j + 2) * cur_val < size):
                    bool_vals[((j + 2) * cur_val) - 2] = False
                    cur_val += 1
                subcycle_lengths.append(j + 2)
            j += 1
        
        if (sum(subcycle_lengths) > size):
            subcycle_lengths.pop()
        
        perm = Permutation.cycle()
        start = 0
        for num in subcycle_lengths:
            perm *= Permutation.cycle(*shuffled_nums[start:start + num])
            start += num
            
        return perm

In [ ]:
# generate matrix permutation - needs to be altered manually before running script for different k-values

p = generate_permutation(3)

In [ ]:
# recover matrix permutation

def permute_list(a):
    b = []
    for i in range(len(a)):
        b.append(a[p(i+1)-1]) ## have to adjust indices
    return b
    
def h(a):
    b = []
    for i in range(len(a)):
        b.append([])
        for j in range(len(a)):
            b[i].append(permute_list(a[i][j]))
    return b

In [ ]:
# generate all test values

def convert_string(i,n):
    """turn a number i into a length n list of true, false values"""
    b = str(bin(i))[2:].zfill(n) ##convert i to binary, remove 0b, pad the front to give appropriate string length
    c = []
    for j in range(n):
        if b[j] == '0': ##needs to be in '' since converted to string
           c.append(False)
        if b[j] == '1':
           c.append(True)
    return c      

def all_lists(n):
    """return all possible length n lists of true, false values"""
    lists = []
    for i in range(2**n):
        lists.append(convert_string(i,n))
    return lists

def pack(a):
    """pack a list into a Boolean matrix. Note single values are held in lists for compatibility with multiplication function"""
    mat = []
    s = floor(sqrt(len(a))) ## floor to remove float from sqrt(len(a)), indexing will fail if len(a) not square
    for i in range(s):
        mat.append([]) ## initialise rows
        for j in range(s):
            mat[i].append([a[i*s+j]])
    return mat

def all_matrices(k):
    """compile a list of all k x k single-bit Boolean matrices"""
    matrices = []
    b = all_lists(k**2)
    for i in range(len(b)):
        matrices.append(pack(b[i]))
    return matrices



In [ ]:
# semidirect product and exponentiation

def compose_permutations(g, h):
    return lambda a : h(g(a))
    
def semidirect_product(tup1, tup2):
    perm_mat = tup2[1](tup1[0])
    first_elem = prod_bool_mat(perm_mat, tup2[0])
    second_elem = compose_permutations(tup1[1],tup2[1])
    return (first_elem, second_elem)
    
def exponentiation(tup,k):
    k_binary = bin(k)
    value = tup
    for digit in range(3,len(k_binary)):
        value = semidirect_product(value, value)[:]
        if (k_binary[digit] == '1'):
           value = semidirect_product(value, tup)[:]
    return value 

In [ ]:
# generate exchange value for test

def generate_A(M,h,x): #M, h need to be predefined
    A = exponentiation((M,h),x)
    return A[0]

    

In [ ]:
#generate all check matrices separately so it only has to be run once. again has to be altered manually before running depending on n value

global mats

mats = all_matrices(2)


In [ ]:
# decompose into direct sum components 

def pull(k,a):
    b = []
    for i in range(len(a)):
        b.append([])
        for j in range(len(a)):
            b[i].append([a[i][j][k]])
    return b

In [ ]:
# count solutions to given equation
    
def count_singlebit_solutions(a, b): ##takes a list of matrices to count solutions from as an input
    count = 0
    for x in mats:
        if a == prod_bool_mat(x, b):
           count = count + 1
        else:
           count = count + 0 
    return count

def count_solutions(a, b):
    count = 1
    for i in range(len(a[0][0])):
        count = count * count_singlebit_solutions(pull(i, a), pull(i, b))
    return count      


In [ ]:
# count size of an orbit

def check_membership(M, values):
    for x in values:
        if M == x:
           return 'True'
    return False   

def count_singlebit_orbit(M):
    orbit = []
    for y in mats:
        if not check_membership(prod_bool_mat(y, M), orbit):
           orbit.append(prod_bool_mat(y, M))
    return len(orbit)       


def count_orbit(M):
    n = len(M)
    k = len(M[0][0])
    orbit_count = 1
    for i in range(k):
        orbit_count = orbit_count * count_singlebit_orbit(pull(i, M))
    return orbit_count

def count_orbit_2(M):
    n = len(M)
    k = len(M[0][0])
    orbit_counts = []
    for i in range(k):
        orbit_counts.append(count_singlebit_orbit(pull(i, M)))
    return orbit_counts        

In [ ]:
def product(lst):
    p = 1
    for i in lst:
        p *= i
    return p

In [ ]:
def truncate(number, digits) -> float:
    stepper = 10.0 ** digits
    return math.trunc(stepper * number) / stepper

In [ ]:
# experiment 1 - varied exponent

M = rand_bool_mat(3, 5)

def count_telescope_solutions_1(mat, mat_perm):
  x = randint(20, 100)
  a = generate_A(M, mat_perm, x)
  b = prod_bool_mat(mat_perm(a), M)
  sols_list = []
  for i in range(len(M[0][0])):
    single_sol = count_singlebit_solutions(pull(i, b), pull(i, a))
    sols_list.append(single_sol)
  return (x, sols_list, product(sols_list))  

  



In [ ]:
# experiment 2 - varied matrix

x = 100

def count_telescope_solutions_2(mat_size, bs_length, mat_perm, exp):
  M = rand_bool_mat(mat_size, bs_length)
  a = generate_A(M, mat_perm, exp)
  b = prod_bool_mat(mat_perm(a), M)
  return M, count_solutions(b, a)

In [ ]:
# experiment 3 - varied matrix, recording orbit size

x = 100

def count_telescope_solutions_3(mat_size, bs_length, mat_perm, exp):
  M = rand_bool_mat(mat_size, bs_length)
  a = generate_A(M, mat_perm, exp)
  b = prod_bool_mat(mat_perm(a), M)
  return count_orbit(a), count_solutions(a, b)

In [ ]:
# experiment 4 - varied matrix, recording orbit size, discard irregular cases

x = 100

def count_telescope_solutions_4(mat_size, bs_length, mat_perm, exp):
  M = rand_bool_mat(mat_size, bs_length)
  a = generate_A(M, mat_perm, exp)
  b = prod_bool_mat(mat_perm(a), M)
  if count_solutions(b, a) != count_solutions(a,b):
    return False, math.log(count_orbit(a)), math.log(count_solutions(b, a))
  else:
    return True, math.log(count_orbit(a)), math.log(count_solutions(b, a))

In [ ]:
outF = open("desired_file_name.txt", "w")
for i in range(100):
  outF.write(str(count_telescope_solutions_4(#n-value, k-value, permutaiton, number of trials)))
  outF.write("\n")
outF.close()



